# Screeners
Some examples on how you can create your own screeners and export txt-files that are importable in TradingView.
This notebook utilizes the same pattern as the breadth-indicator one; big dataframe with unique identifiers into groupy -> apply for data manipulation and some pandas magic to get the data out.
First we will create some helper functions to be able to create TradingView-compatible tickers, calculate some momentum parameters, earnings and revenue growth and turnover (SEK).
These parameters will then be used in various way to generate importable text-files for TradingView.

We will utilize stored pickles for the data and the dataframes can grow quite large hence it'll be kinda heavy on RAM.

Let's go!

In [32]:
# imports
import csv
import constants
import pandas as pd 
import os
import numpy as np

# pandas options for string representation of data frames (print)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def create_trading_view_watchlist(df: pd.DataFrame, filename: str):
     tv_tickers = []
     for index, row in df.iterrows():
          ticker = row.ticker.replace(" ", "_")
          market = row.market.lower()
          country = row.country.lower()
          ticker_tv = ""
          if market == "spotlight" or market == "ngm":
               ticker_tv = "ngm:" + ticker
          elif country == "sverige":
               ticker_tv = "omxsto:" + ticker
          elif country == "finland":
               ticker_tv = "omxhex:" + ticker
          elif country == "danmark":
               ticker_tv = "omxcop:" + ticker
          elif country == "norge":
               ticker_tv = "osl:" + ticker
          tv_tickers.append(ticker_tv)
     with open(constants.EXPORT_PATH + filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(tv_tickers)
        print(f"Watchlist created: {constants.EXPORT_PATH + filename}")

def calculate_momentum_and_turnover(df: pd.DataFrame) -> pd.DataFrame:
    # net up daus is also a fun and seldomly used momentum indicator.
    df["net_up_days_252"] = (df['close'].pct_change() >= 0).rolling(252).sum()
    df["abs65"] = df["close"]/df["close"].rolling(65).min()
    df["abs125"] = df["close"]/df["close"].rolling(125).min()
    df["oneil_rs"] = df["close"].pct_change(65)*2 + df["close"].pct_change(125) + df["close"].pct_change(189) + df["close"].pct_change(252)
    df["turnover"] = df["close"]*df["volume"] if df["country"].values[0] != "finland" else df["close"]*df["volume"]*10 # simple conversion for EUR (skipping other currency diffs)
    df["turnover_sma50"] = df["turnover"].rolling(50).mean()
    return df

# read stored stock data from pickle if it exists else create it
if os.path.isfile(constants.EXPORT_PATH + "data.pickle"):
     symbols = pd.read_pickle(constants.EXPORT_PATH + "data.pickle")
else:
     import borsdata_client as bc
     client = bc.BorsdataClient()
     symbols = client.get_all_stock_data_and_save_to_disk()
# calculate momentum parameters for each df
symbols = symbols.groupby("stock_id", as_index=False).apply(lambda x: calculate_momentum_and_turnover(x))
symbols.set_index("date", inplace=True)
# get top 100 stock sorted by absolute momentum 65 days and some liquidity filter and create a watchlist for TradingView.
abs65 = symbols[(symbols.index == symbols.last_valid_index()) & (symbols["turnover_sma50"] > 5e6)].sort_values("abs65").tail(100).copy()
create_trading_view_watchlist(abs65, "abs65_top_100_turnover_greater_than_5e6.txt")

Watchlist created: file_exports/abs65_top_100_turnover_greater_than_5e6.txt


## Report data

So what if we wanted to get report data into the equation... My personal favorites are growth of earnings and revenues.

In [33]:
import numpy as np
def calc_pct_change(end, start):
    if end <= 0 or end == float("inf") or start == float("inf") or start == 0:
        return np.nan
    return (end - start) / abs(start)

def calc_pct_change_series(column, shift):
    ret_val = np.nan
    try:
        ret_val = column.pct_change(shift) * np.sign(column.shift(shift))
    except:
        pass
    return ret_val

def calculate_growth_quarterly(reports_quarter: pd.DataFrame) -> pd.DataFrame:
    reports_quarter.sort_values("report_date", inplace=True)
    reports_quarter['net_profit_margin'] = reports_quarter['profit_to_equity_holders'] / \
        reports_quarter['revenues']

    for _index, row in reports_quarter.iterrows():
        current_rev = row['revenues']
        current_eps = row['earnings_per_share']
        current_npm = row['net_profit_margin']
        year = row['year']
        period = row['period']
        try:
            prev_year = year - 1
            prev_eps = reports_quarter.loc[(reports_quarter['year'] == prev_year) & (
                reports_quarter['period'] == period)].earnings_per_share.values[0]
            prev_rev = reports_quarter.loc[(reports_quarter['year'] == prev_year) & (
                reports_quarter['period'] == period)].revenues.values[0]
            prev_npm = reports_quarter.loc[(reports_quarter['year'] == prev_year) & (
                reports_quarter['period'] == period)].net_profit_margin.values[0]
            reports_quarter.loc[(reports_quarter['year'] == year) & (
                reports_quarter['period'] == period), 'eps_growth_quarter_yy'] = calc_pct_change(
                current_eps, prev_eps)
            reports_quarter.loc[(reports_quarter['year'] == year) & (
                reports_quarter['period'] == period), 'rev_growth_quarter_yy'] = calc_pct_change(
                current_rev, prev_rev)
            reports_quarter.loc[(reports_quarter['year'] == year) & (
                reports_quarter['period'] == period), 'npm_growth_quarter_yy'] = calc_pct_change(
                current_npm, prev_npm)

        except Exception as e:
            reports_quarter.loc[(reports_quarter['year'] == year) & (
                reports_quarter['period'] == period), 'eps_growth_quarter_yy'] = np.nan
            reports_quarter.loc[(reports_quarter['year'] == year) & (
                reports_quarter['period'] == period), 'rev_growth_quarter_yy'] = np.nan
            reports_quarter.loc[(reports_quarter['year'] == year) & (
                reports_quarter['period'] == period), 'npm_growth_quarter_yy'] = np.nan
    return reports_quarter

def calculate_growth_yearly(reports_year):
    reports_year.sort_values("report_date", inplace=True)
    reports_year['eps_growth1'] = calc_pct_change_series(
        reports_year['earnings_per_share'], 1)
    reports_year['eps_growth2'] = calc_pct_change_series(
        reports_year['earnings_per_share'], 2)
    reports_year['eps_growth3'] = calc_pct_change_series(
        reports_year['earnings_per_share'], 3)
    reports_year['rev_growth1'] = calc_pct_change_series(
        reports_year['revenues'], 1)
    reports_year['rev_growth2'] = calc_pct_change_series(
        reports_year['revenues'], 2)
    reports_year['rev_growth3'] = calc_pct_change_series(
        reports_year['revenues'], 3)
    reports_year['net_profit_margin'] = reports_year['profit_to_equity_holders'] / \
        reports_year['revenues']
    return reports_year
            
# read stored stock data from pickle if it exists else create it
if os.path.isfile(constants.EXPORT_PATH + "reports_quarter_data.pickle") and os.path.isfile(constants.EXPORT_PATH + "reports_year_data.pickle") :
    reports_quarter = pd.read_pickle(constants.EXPORT_PATH + "reports_quarter_data.pickle")
    reports_year = pd.read_pickle(constants.EXPORT_PATH + "reports_year_data.pickle")
else:
    import borsdata_client as bc
    client = bc.BorsdataClient()
    client.get_all_report_data_and_save_to_disk()

if not os.path.isfile(constants.EXPORT_PATH + "reports_quarter_growth.pickle"):
    reports_quarter = pd.read_pickle(constants.EXPORT_PATH + "reports_quarter_data.pickle")
    reports_quarter["report_date"] = pd.to_datetime(reports_quarter["report_date"])
    # calculate growth parameters for the report data
    reports_quarter = reports_quarter.groupby("stock_id", as_index=False).apply(lambda x: calculate_growth_quarterly(x))
    # store it if we want to pick it up later (the calculation is a bit heavy and takes ~5-6 minutes)
    reports_quarter.to_pickle(constants.EXPORT_PATH + "reports_quarter_growth.pickle")
else:
    reports_quarter = pd.read_pickle(constants.EXPORT_PATH + "reports_quarter_growth.pickle")

if not os.path.isfile(constants.EXPORT_PATH + "reports_year_growth.pickle"):
    reports_year = pd.read_pickle(constants.EXPORT_PATH + "reports_year_data.pickle")
    reports_year["report_date"] = pd.to_datetime(reports_year["report_date"])
    # calculate growth parameters for the report data
    reports_year = reports_year.groupby("stock_id", as_index=False).apply(lambda x: calculate_growth_yearly(x))
    # store it if we want to pick it up later (the calculation is a bit heavy and takes ~5-6 minutes)
    reports_year.to_pickle(constants.EXPORT_PATH + "reports_year_growth.pickle")
else:
    reports_year = pd.read_pickle(constants.EXPORT_PATH + "reports_year_growth.pickle")

Now we have price data and report data available. It is time to combine it to a new dataframe. We will loop through all the values for the last valid index in the price data and fetch the report data that we are interested in and combine it into a new dataframe.

In [34]:
stock_dict_list = []
for index, row in symbols[(symbols.index == symbols.last_valid_index())].iterrows():
    stock_id = row["stock_id"]
    try:
        # get the reports for this stock
        reports_q = reports_quarter[reports_quarter.stock_id == stock_id]
        reports_y = reports_year[reports_year.stock_id == stock_id]
        # last value == latest, easiest way to get it is by indexing "backwards" -1 represents the last element
        # -2 the second last etc.
        # eps data
        last_reported_eps_growth = reports_q["eps_growth_quarter_yy"].values[-1] 
        second_last_reported_eps_growth = reports_q["eps_growth_quarter_yy"].values[-2]
        last_reported_eps_growth_y = reports_y["eps_growth1"].values[-1]
        # revenue data
        last_reported_rev_growth = reports_q["rev_growth_quarter_yy"].values[-1] 
        second_last_reported_rev_growth = reports_q["rev_growth_quarter_yy"].values[-2]
        last_reported_rev_growth_y = reports_y["rev_growth1"].values[-1]
        # append a new dict-object to the list
        stock_dict_list.append({"name": row["name"],"ticker": row["ticker"], "market": row["market"], "country": row["country"], 
                                "abs65": row["abs65"], "abs125": row["abs125"], "oneil_rs": row["oneil_rs"], "net_up_days_252": row["net_up_days_252"],
                                "eps_growth_q1_yy": last_reported_eps_growth, 
                                "eps_growth_q2_yy": second_last_reported_eps_growth,
                                "eps_growth_1y": last_reported_eps_growth_y, 
                                "rev_growth_q1_yy": last_reported_rev_growth,
                                "rev_growth_q2_yy": second_last_reported_rev_growth,
                                "rev_growth_1y": last_reported_rev_growth_y,
                                "turnover_sma50": row["turnover_sma50"]})
    except Exception as e:
        # for some companies the data won't be available. Just pass the error and get to the next.
        pass

# create a the new dataframe from the list of dictionaries.
momo_and_growth = pd.DataFrame(stock_dict_list)
# print the result
print(momo_and_growth.tail())

                   name ticker     market country     abs65    abs125   
1659   Sparebanken Vest   SVEG  Oslo Bors   Norge  1.077465  1.115431  \
1660  Sandnes Sparebank   SADG  Oslo Bors   Norge  1.009238  1.009238   
1661   Totens Sparebank   TOTG  Oslo Bors   Norge  1.019802  1.131868   
1662    Sparebanken Sor    SOR  Oslo Bors   Norge  1.000000  1.024691   
1663  Aurskog Sparebank   AURG  Oslo Bors   Norge  1.000000  1.009615   

      oneil_rs  net_up_days_252  eps_growth_q1_yy  eps_growth_q2_yy   
1659  0.065752            132.0         -0.106697          0.341398  \
1660 -0.344683            133.0         -0.332360         -0.182476   
1661 -0.167826            164.0          0.118238          0.063964   
1662 -0.370788            140.0          0.429874         -0.152867   
1663 -0.001882            188.0          1.148324          0.145225   

      eps_growth_1y  rev_growth_q1_yy  rev_growth_q2_yy  rev_growth_1y   
1659       0.244928          0.030984          0.276687     

We now have a dataframe containing both the price data and growth parameters for the last two reported quarters and the last year for earnings and revenues. Lets create some new watchlists (screeners) from the newly created data and export the dataframe to excel for all the excel-crunchers out there.

In [35]:
# save to excel
momo_and_growth.to_excel(constants.EXPORT_PATH + "momo_and_growth.xlsx")
# eps and revenues growth > 0% last two quarters and last year, turnover last 50 day average greater than 5 million SEK
temp = momo_and_growth[(momo_and_growth["eps_growth_q1_yy"] > 0) &
                       (momo_and_growth["eps_growth_q2_yy"] > 0) &
                       (momo_and_growth["rev_growth_q1_yy"] > 0) & 
                       (momo_and_growth["rev_growth_q2_yy"] > 0) & 
                       (momo_and_growth["eps_growth_1y"] > 0) & 
                       (momo_and_growth["rev_growth_1y"] > 0) & 
                       (momo_and_growth["turnover_sma50"] > 5e6)]
create_trading_view_watchlist(temp, "eps_and_rev_growth.txt")

Watchlist created: file_exports/eps_and_rev_growth.txt


Another quite common approach while screening is ranking. Lets rank on the momentum attributes...

In [36]:
momo_and_growth["abs65_rank"] = momo_and_growth["abs65"].rank(ascending=True, pct=True).round(2) * 100
momo_and_growth["abs125_rank"] = momo_and_growth["abs125"].rank(ascending=True, pct=True).round(2) * 100
# this is the infamous "RS-Rank"
momo_and_growth["rs"] = momo_and_growth["oneil_rs"].rank(ascending=True, pct=True).round(2) * 100

# eps and revenues growth > 0% last quarter and turnover last 50 day average greater than 5 million SEK and abs65-rank or abs125-rank greater than 90 (top 10% perfomer)
temp = momo_and_growth[(momo_and_growth["eps_growth_q1_yy"] > 0) & (momo_and_growth["rev_growth_q1_yy"] > 0) & (momo_and_growth["turnover_sma50"] > 5e6) & ((momo_and_growth["abs65_rank"] > 90) | ((momo_and_growth["abs125_rank"] > 90)))]
create_trading_view_watchlist(temp, "abs65_or_abs125_rank_greater_than_90_and_eps_and_rev_growth.txt")

temp = momo_and_growth[(momo_and_growth["eps_growth_q1_yy"] > 0) & (momo_and_growth["rev_growth_q1_yy"] > 0) & (momo_and_growth["rs"] > 90) & (momo_and_growth["turnover_sma50"] > 5e6)]
create_trading_view_watchlist(temp, "rs_rank_greater_than_90_and_eps_and_rev_growth.txt")

Watchlist created: file_exports/abs65_or_abs125_rank_greater_than_90_and_eps_and_rev_growth.txt
Watchlist created: file_exports/rs_rank_greater_than_90_and_eps_and_rev_growth.txt


This was all, I hope you'll find it useful and educative. If any questions arise feel free to contact me on [Twitter](https://twitter.com/TapeReaderJoe).

And of course there might be some errors in these calculations and the code can hopefully been written nicer!